In [1]:
import tweepy
import sys
import threading, apscheduler.schedulers.background
import time
from textblob import TextBlob

class MyStreamListener(tweepy.StreamListener):

    def __init__(self, c, t_interval):
        tweepy.StreamListener.__init__(self)
        #self.counts = dict()
        self.counts_pos = dict()
        self.counts_neg = dict()
        self.c = c
        self.t_interval = t_interval 
        
    def on_status(self, status):
        mentions = status.entities['user_mentions']
        print(status.text)
        if len(mentions) >= 0:      
            polarity = TextBlob(status.text).sentiment.polarity
            print('polarity: ' + str(polarity))        
            if(polarity >= 0):
                delete = []    
                for user in self.counts_pos:
                    if self.counts_pos[user] < 0.5:
                        delete.append(user)
                    else:
                        self.counts_pos[user] *= (1-self.c) 
                for user in delete: 
                    del self.counts_pos[user] 

                for mention in mentions:
                    self.counts_pos[mention['name']] = self.counts_pos.get(mention['name'], 0) + 1    
            else:
                delete = []    
                for user in self.counts_neg:
                    if self.counts_neg[user] < 0.5:
                        delete.append(user)
                    else:
                        self.counts_neg[user] *= (1-self.c) 
                for user in delete: 
                    del self.counts_neg[user] 

                for mention in mentions:
                    self.counts_neg[mention['name']] = self.counts_neg.get(mention['name'], 0) + 1  
            
    def on_timeout(self):
        print('timeout')
        return False
        
    def on_error(self, status_code):
        print('Error: ' + str(status_code))
        with open("PopularUsers.txt", "a", encoding="utf-8") as text_file:
            print('Error: ' + str(status_code), file=text_file) 
            
        if status_code == 420:    
            print('Disconected: Rate limit surpassed: ' + str(status_code))
            #returning False in on_error disconnects the stream
            return False
        else:
            return True
            # returning non-False reconnects the stream, with backoff.

In [2]:
def getPopularUsers(myStreamListener):
    result = ""
    for k, v in sorted(myStreamListener.counts_pos.items(), key=lambda item: item[1], reverse=True)[:10]:
        result += k + ' (' + str(v) + '); '
    print('Popular: ' + result)
    with open("PopularUsers.txt", "a", encoding="utf-8") as text_file:
        print(result, file=text_file)   
    
    result = ""
    for k, v in sorted(myStreamListener.counts_neg.items(), key=lambda item: item[1], reverse=True)[:10]:
        result += k + ' (' + str(v) + '); '
    print('Unpopular: ' + result)
    with open("unPopularUsers.txt", "a", encoding="utf-8") as text_file:
        print(result, file=text_file)  

In [3]:
if len(sys.argv) < 2:
    print("I am Missing some arguments (<time interval T>")
    exit(-1)
else:
    t_interval = sys.argv[1]
    if not t_interval.isnumeric():
        t_interval = 10.0
        
c = 1E-2

In [4]:
auth = tweepy.OAuthHandler('yoJRxZxTqy1rCTSysDU17kaPP', 'IU6g1kGSshXp78xbFa9bcgDNhC9iqiLOu0TJGuHFva1IUHcFSQ')
auth.set_access_token('446228134-EceCoQoNc6Xe2U0NhN3Xd0qU4SvM39xSZ373YAp6', '8qW0JSQ7KOZweRnQIcYnj9wgVzGSZ8As1BVGNuYlygQLi')
twAPI = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

global myStreamListener

print('Connecting to Twitter..')
myStreamListener = MyStreamListener(c, t_interval)
myStream = tweepy.Stream(auth = twAPI.auth, listener=myStreamListener)
myStream.filter(locations=[-8.6, 49.5, 1.46, 60.5], is_async=True) #UK
print('Connected.')

scheduler = apscheduler.schedulers.background.BackgroundScheduler()
scheduler.add_job(getPopularUsers, 'interval', args = [myStreamListener], seconds=(int) (t_interval))
scheduler.start()
print('Updating 10 most popular users on Twitter in UK:')

Connecting to Twitter..
Connected.
Updating 10 most popular users on Twitter in UK:
